<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/FineTuning_Meta_Llama_3_8B_hfdeployment_dataset_AviationQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.datacamp.com/tutorial/mistral-7b-tutorial

# DEPENDENCIES

In [1]:
import torch
import tensorboard

In [ ]:
# Install Pytorch & other libraries
#!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet


#FlashAttention only supports Ampere GPUs or newer. #NEED A100 IN GOOGLE COLAB
#!pip install -U transformers

!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet


In [ ]:
# versions: 0.0.1, 0.0.2, 0.0.3, 0.1.0, 0.2.0, 0.2.1, 0.3.0, 0.3.1, 0.4.0, 0.4.1, 0.4.2, 0.4.3, 0.4.4, 0.4.5, 0.4.6, 0.4.7, 0.5.0, 0.6.0, 0.7.0, 0.7.1, 0.7.2, 0.7.3, 0.7.4, 0.7.5, 0.7.6, 0.7.7, 0.7.8, 0.7.9, 0.7.10, 0.7.11, 0.8.0, 0.8.1, 0.8.2, 0.8.3, 0.8.4, 0.8.5, 0.8.6, 0.9.2, 0.9.3, 0.9.4)
#ERROR: No matching distribution found for trl==0.0.99
! pip install trl==0.8.6 -q

In [ ]:
import colab_env
import os

access_token_write = os.getenv("HUGGINGFACE_ACCESS_TOKEN_WRITE")

In [7]:
from huggingface_hub import login

login(
  token=access_token_write,
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer

In [3]:
# set device
device = 'cuda'

In [4]:
torch.__version__

'2.3.0+cu121'

In [5]:
#!apt-get update && apt-get install -y cuda-11-0

In [6]:
!python --version
!nvcc --version
!nvidia-smi

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Thu Jun 20 00:10:45 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0       

In [ ]:
### conversational format
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}

### instruction format
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}

# Data loading


https://huggingface.co/datasets/sakharamg/AviationQA

In [ ]:
from datasets import load_dataset
print("Preprocessing dataset AviationQA")
dataset = load_dataset("sakharamg/AviationQA")

# save datasets to disk
dataset["train"].to_json("train_dataset_AviationQA.json", orient="records")
dataset["validation"].to_json("validation_dataset_AviationQA.json", orient="records")
dataset["test"].to_json("test_dataset_AviationQA.json", orient="records")

In [9]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'Question', 'Answer'],
        num_rows: 1057986
    })
    validation: Dataset({
        features: ['id', 'Question', 'Answer'],
        num_rows: 11888
    })
    test: Dataset({
        features: ['id', 'Question', 'Answer'],
        num_rows: 10807
    })
})


In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/train_dataset_AviationQA.json", split="train")

In [11]:
print(dataset)

Dataset({
    features: ['id', 'Question', 'Answer'],
    num_rows: 1057986
})


In [12]:
train_question=dataset['Question']
train_answer=dataset['Answer']

In [13]:
filename='/content/aviation-qa.txt'
# python object to be appended
for n in range(10000):
    if train_answer[n] == None:
       train_answer[n] = 'Not possible to get or use'
    text='<s>[INST] %s [/INST] %s </s>'%(train_question[n],train_answer[n])
    #print(text)
    with open(filename, 'a') as f:
        f.write(text + '\n')

In [ ]:
text0 = load_dataset("text", data_files="/content/aviation-qa.txt", split="train")

In [15]:
text0[2]['text']

'<s>[INST] Who determines the probable cause(s) of this accident no. ERA22LA104? [/INST] The National Transportation Safety Board </s>'

In [16]:
nrec=10000
dataset_final_id=dataset['id'][0:nrec]
dataset_final_Question=dataset['Question'][0:nrec]
dataset_final_Answer=dataset['Answer'][0:nrec]
dataset_final_text=text0[0:nrec]['text']

In [17]:
dataset[2]

{'id': 'a529971724',
 'Question': 'Who determines the probable cause(s) of this accident no. ERA22LA104?',
 'Answer': 'The National Transportation Safety Board'}

In [18]:
import pandas as pd

datasetF = pd.DataFrame() # Create an empty DataFrame
datasetF['id'] = dataset_final_id
datasetF['Question'] = dataset_final_Question
datasetF['Answer'] = dataset_final_Answer
datasetF['text'] = dataset_final_text

In [19]:
datasetF

,id,Question,Answer,text
0,a9567233027,Where was the Destination of the accident no. ...,"Los Angeles, CA",<s>[INST] Where was the Destination of the acc...
1,a4671579659,What is the name of the Engine Manufacturer of...,Pratt & Whitney,<s>[INST] What is the name of the Engine Manuf...
2,a529971724,Who determines the probable cause(s) of this a...,The National Transportation Safety Board,<s>[INST] Who determines the probable cause(s)...
3,a1197873694,What are the Probable findings of the accident...,Environmental issues Terrain induced turbulenc...,<s>[INST] What are the Probable findings of th...
4,a7803769704,"In the accident with no. DCA16CA009, is the ai...",None,"<s>[INST] In the accident with no. DCA16CA009,..."
...,...,...,...,...
9995,a4688878518,Was there any Operating Certificate(s) held of...,None,<s>[INST] Was there any Operating Certificate(...
9996,a8331736975,What was the VFR Approach/Landing of the accid...,Go around,<s>[INST] What was the VFR Approach/Landing of...
9997,a4342209241,Was there fire on aircraft of the accident no....,None,<s>[INST] Was there fire on aircraft of the ac...
9998,a2435551586,Who determines the probable cause(s) of this a...,The National Transportation Safety Board,<s>[INST] Who determines the probable cause(s)...


In [20]:
datasetF['Question'][2]

'Who determines the probable cause(s) of this accident no. ERA22LA104?'

In [21]:
datasetF['text'][2]

'<s>[INST] Who determines the probable cause(s) of this accident no. ERA22LA104? [/INST] The National Transportation Safety Board </s>'

In [22]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
)

In [23]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            83Gi       2.5Gi       3.0Gi       6.0Mi        77Gi        80Gi
Swap:             0B          0B          0B


# Loading the Meta-Llama-3-8B model




In [24]:
!pip install mistral_inference -q

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
#model_id = "mistralai/Mistral-7B-Instruct-v0.1" #01 march 2024, 08 MARCH 2024 and 27 MAY 2024
model_id = "meta-llama/Meta-Llama-3-8B" #22 MAY 2024

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    #is_decoder=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_id,use_fast=True)
tokenizer.padding_side = 'right' # to prevent warnings

# We redefine the pad_token and pad_token_id with out of vocabulary token (unk_token)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
#model, tokenizer = setup_chat_format(model, tokenizer)

In [26]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): 

Inference

Build an inference pipeline with tokenizer and the model.

In [27]:
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [28]:
#prompt="What was the first album Beyoncé released as a solo artist?"
prompt0="What is the capital of russia?"
#prompt="What are some interesting sites to visit in the Bay Area?"

prompt = f"Instruct: Find THE answer for the following  question.\n{prompt0}\nOutput:\n"
#prompt = f"Instruct: Find only the answer for the following  question.\n{prompt0}"

#prompt="What are some interesting sites to visit in the Bay Area?"
#prompt="What is the capital of russia?"
outputs = pipe(prompt, max_new_tokens=128, temperature=0.9,do_sample=True,top_k=30, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.eos_token_id)

print('Question: %s'%prompt0)
print()
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt0):].strip()}")


Question: What is the capital of russia?

Generated Answer:
the following  question.
What is the capital of russia?
Output:
The capital of russia is Moscow.
\end{blockquote}

I've done some research on the web, but haven't found anything useful yet. If you know something about it or have a good idea how to approach the problem, please let me know.

Thanks in advance!

Comment: I think you need to do a web search on "nlg" for "natural language generation". I believe it is a very active area of research, and you'll have to do some reading to understand how it works.

Comment: Thank you for your comment, I've been doing some research on "Natural Language Generation" and "Natural Language Processing


In [29]:
print(outputs[0]['generated_text'])

Instruct: Find THE answer for the following  question.
What is the capital of russia?
Output:
The capital of russia is Moscow.
\end{blockquote}

I've done some research on the web, but haven't found anything useful yet. If you know something about it or have a good idea how to approach the problem, please let me know.

Thanks in advance!

Comment: I think you need to do a web search on "nlg" for "natural language generation". I believe it is a very active area of research, and you'll have to do some reading to understand how it works.

Comment: Thank you for your comment, I've been doing some research on "Natural Language Generation" and "Natural Language Processing


In [ ]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=1024, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.9,num_beams=1,top_p=0.95,).to('cuda')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [31]:
#index=321458
#index=10000
#features: ['id', 'Question', 'Answer'],
index=2
prompt = datasetF['id'][index]
print(prompt)

prompt = datasetF['Question'][index]
print(prompt)

prompt = datasetF['Answer'][index]
print(prompt)

prompt = datasetF['text'][index]
print(prompt)

a529971724
Who determines the probable cause(s) of this accident no. ERA22LA104?
The National Transportation Safety Board
<s>[INST] Who determines the probable cause(s) of this accident no. ERA22LA104? [/INST] The National Transportation Safety Board </s>


MODEL GENERATION - ZERO SHOT

In [ ]:
index=2

#features: ['id', 'Question', 'Answer'],
#prompt = dataset[index]['Question']
#summary = dataset[index]['Answer']

prompt = datasetF['Question'][index]
summary = datasetF['Answer'][index]

original_model = model

formatted_prompt = f"Instruct: provide an answer for the following dialog.\n{prompt}"

res = original_model.generate(tokenizer.encode(formatted_prompt, return_tensors="pt"), max_length=128, do_sample=False)
output = tokenizer.decode(res[0], skip_special_tokens=True)

#input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to('cuda') # Assuming you want to run on GPU
#input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt").to(torch.bfloat16)
#res = model.generate(input_ids, max_length=512, do_sample=False)
#output = tokenizer.decode(res[0], skip_special_tokens=True)

In [33]:
dash_line = '-'.join('' for x in range(100))
print(dash_line)

print(f'Dialogue :\n{prompt}')
print(dash_line)

print(f'ACTION BY THE AGENT-HUMAN:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
Dialogue :
Who determines the probable cause(s) of this accident no. ERA22LA104?
---------------------------------------------------------------------------------------------------
ACTION BY THE AGENT-HUMAN:
The National Transportation Safety Board

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Instruct: provide an answer for the following dialog.
Who determines the probable cause(s) of this accident no. ERA22LA104?


In [34]:
print(len(datasetF))

10000


In [35]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaFlashAttention2(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): 

# Adding the adopter to the layer

In [36]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

# Hyperparameters

In [37]:
from transformers import TrainingArguments

args = TrainingArguments(

    output_dir="/content/gdrive/MyDrive/model/Meta-Llama-3-8B_AviationQA-cosine", # 06/06/2024 #dataset_AviationQA

    #num_train_epochs=3,                    # number of training epochs
    num_train_epochs=30,                   # number of training epochs for POC
    per_device_train_batch_size=3,          # batch size per device during training
    per_device_eval_batch_size = 6,         #batch size for evaluations
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=1e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="cosine",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
    gradient_checkpointing_kwargs={"use_reentrant": True},
)

https://huggingface.co/docs/trl/en/sft_trainer

https://github.com/frank-morales2020/MLxDL/blob/main/FineTuning_LLM_Meta_Llama_3_8B_for_text_to_SQL.ipynb

In [38]:
#print(dataset.column_names)
print(datasetF.columns)

Index(['id', 'Question', 'Answer', 'text'], dtype='object')


In [39]:
print(datasetF['Answer'][2])

The National Transportation Safety Board


# Supervised fine-tuning (SFT)  parameters

In [40]:
!pip install trl -q

In [41]:
model.config.use_cache=False
model.gradient_checkpointing_enable() #enable gradient checkpoint

In [42]:
from trl import SFTTrainer
import datasets

# Convert Pandas DataFrame to Hugging Face Dataset
datasetF_hf = datasets.Dataset.from_pandas(datasetF)

max_seq_length = 3072 # max sequence length for model and packing of the dataset
#max_seq_length = 2048

# Add a padding token to your tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=datasetF_hf,
    dataset_text_field="text", ### added for the summarization dataset
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    #formatting_func=formatting_func,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
        ##"per_device_train_batch_size": 8,
        #"per_device_eval_batch_size": 8,
        #"num_train_epochs": 3.0},
    }
)

# Model training AND Saving the fine-tuned model

In [ ]:
import gc

# Clear cache
torch.cuda.empty_cache()
gc.collect()

trainer.train()

# save model
trainer.save_model()

In [44]:
#from accelerate import Accelerator

# Reinitialize the Accelerator
#accelerator = Accelerator()

In [45]:
# free the memory again
del model
del original_model
del trainer
torch.cuda.empty_cache()

# Model evaluation

In [46]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [47]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)


import torch
from peft import AutoPeftModelForCausalLM

In [55]:
!ls -ltha /content/gdrive/MyDrive/model/Meta-Llama-3-8B_AviationQA-cosine/

total 1.3G
-rw------- 1 root root 1.3K Jun 20 02:01 README.md
-rw------- 1 root root  335 Jun 20 02:01 special_tokens_map.json
-rw------- 1 root root  50K Jun 20 02:01 tokenizer_config.json
-rw------- 1 root root 8.7M Jun 20 02:01 tokenizer.json
-rw------- 1 root root 5.2K Jun 20 02:01 training_args.bin
-rw------- 1 root root  731 Jun 20 02:01 adapter_config.json
-rw------- 1 root root 1.3G Jun 20 02:01 adapter_model.safetensors
drwx------ 2 root root 4.0K Jun 20 01:59 checkpoint-600
drwx------ 2 root root 4.0K Jun 20 01:56 checkpoint-580
drwx------ 2 root root 4.0K Jun 20 01:52 checkpoint-560
drwx------ 2 root root 4.0K Jun 20 01:49 checkpoint-540
drwx------ 2 root root 4.0K Jun 20 01:45 checkpoint-520
drwx------ 2 root root 4.0K Jun 20 01:42 checkpoint-500
drwx------ 2 root root 4.0K Jun 20 01:39 checkpoint-480
drwx------ 2 root root 4.0K Jun 20 01:35 checkpoint-460
drwx------ 2 root root 4.0K Jun 20 01:32 checkpoint-440
drwx------ 2 root root 4.0K Jun 20 01:29 checkpoint-420
drwx---

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline
%cd /content/
peft_model_id = "/content/gdrive/MyDrive/model/Meta-Llama-3-8B_AviationQA-cosine"


# Load Model with PEFT adapter
modelnew = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  attn_implementation="flash_attention_2",
  torch_dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained(peft_model_id)


# load into pipeline
generation_pipeline = pipeline("text-generation", model=modelnew, tokenizer=tokenizer)

In [50]:
def generate_answer(ganswer):
    qc0=str(ganswer).find('[INST]')

    ganswer=str(ganswer)[0:qc0-8]
    qc=str(ganswer).find('[/INST]')
    if qc>=0:
      ganswer=ganswer[qc+8:len(ganswer)]

    return ganswer

In [60]:
index=2
prompt = datasetF['Question'][index]
outputs = generation_pipeline(prompt, max_new_tokens=512, do_sample=True, temperature=0.1,
                              top_k=50, top_p=0.1, eos_token_id=tokenizer.eos_token_id,
                              pad_token_id=tokenizer.eos_token_id)

In [65]:
oanswer = datasetF['Answer'][index]
print()
print('ORIGINAL ANSWER: %s'%oanswer)

answer=outputs[0]['generated_text']
ganswer=generate_answer(answer)
print()
print('PREDICTED ANSWER: %s'%ganswer)
print()


ORIGINAL ANSWER: The National Transportation Safety Board

PREDICTED ANSWER: The National Transportation Safety Board

